This project focuses on building a facial verification app using TensorFlow and Kivy

More specifically, we are creating a Siamese Neural Network for One-Shot Image Recognition

In [1]:
!pip install tensorflow-gpu==2.8.0
!pip install opencv-python matplotlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Check if tf is installed

In [2]:
import tensorflow as tf

print(tf.__version__)
print(tf.test.gpu_device_name())


2.8.0
/device:GPU:0


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
import os


In [5]:
# We can avoid Out of Memory errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
for gpu in gpus:
  print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [7]:
# set up paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [13]:
# Make the directories. Can be found in 'data' folder if running in Colab
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

FileExistsError: ignored

Untar Labelled Faces in the Wild Dataset
http://vis-www.cs.umass.edu/lfw/

In [16]:
# Uncompress Tar GZ Labelled Faces in the wild
tar_path = '/content/drive/MyDrive/facial_recognition/lfw.tgz'

import tarfile

tar = tarfile.open(tar_path, 'r')

# Extract the contents of the tar file to a specific directory
tar.extractall()

# Close the tar file
tar.close()


In [17]:
!ls

data  drive  lfw  sample_data  your_colab_directory_name


In [25]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)


In [44]:
# This script will collect positive and anchor classes and save the image to the respective folder
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import os
from datetime import datetime

def take_photo(label):
  js = Javascript('''
    async function takePhoto(label) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;

      const keydownHandler = async (event) => {
        if (event.key == 'p' || event.key == 'a') {
          div.remove();
          video.pause();
          const ctx = canvas.getContext('2d');
          ctx.drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          const data = canvas.toDataURL('image/jpeg', quality=0.8);
          const response = await fetch(data);
          const blob = await response.blob();
          const reader = new FileReader();
          reader.readAsDataURL(blob);
          reader.onloadend = function() {
            const base64data = reader.result;
            google.colab.kernel.invokeFunction('notebook.TakePhoto', [event.key, base64data], {});
          }
          document.removeEventListener('keydown', keydownHandler);
        }
      }
      document.addEventListener('keydown', keydownHandler);
    }
    ''')
  display(js)
  eval_js('takePhoto({})'.format(label))

def handle_photo(key, data):
  binary = b64decode(data.split(',')[1])
  if key == 'p':
    path = '/content/data/positive'
  elif key == 'a':
    path = '/content/data/anchor'
  if not os.path.exists(path):
    os.makedirs(path)
  # Include timestamp in the filename
  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
  with open(os.path.join(path, f'photo_{timestamp}.jpg'), 'wb') as f:
    f.write(binary)

# Register the function as a callback
google.colab.output.register_callback('notebook.TakePhoto', handle_photo)

# Use the function
take_photo('label')

NameError: ignored

data  datavideo.mp4  drive  lfw  sample_data  your_colab_directory_name
